# Partie A

In [1]:
import yfinance as yf
import pandas as pd

with open('/content/sample_data/Natixis stock (dataset TD1&2).txt', 'r') as file:
  lines = file.readlines()


dates =[]
prices = []

for line in lines:
  line=line.strip()
  if line:
    date,price= line.split('\t')
    dates.append(date)
    prices.append(price)

d = {'Date':dates, 'Price':prices}
df=pd.DataFrame(d)

df['Price']= df['Price'].str.replace(',','.').astype(float)
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df.set_index('Date', inplace=True)

start_date= pd.to_datetime('01/01/2015', format='%d/%m/%Y')
end_date = pd.to_datetime('31/12/2016', format='%d/%m/%Y')

df = df.loc[start_date:end_date]
df

,Price
Date,
2015-01-02,5.621
2015-01-05,5.424
2015-01-06,5.329
2015-01-07,5.224
2015-01-08,5.453
...,...
2016-12-23,5.376
2016-12-27,5.380
2016-12-28,5.379


In [3]:
returns =[]
for i in range(1,len(df)):
  price_yesterday= df['Price'][i-1]
  price_today = df['Price'][i]
  returns.append((price_today-price_yesterday)/price_yesterday)

df['Price Returns'] =  [None] + returns
df = df.iloc[1:] #del the Nan values of the first row



In [24]:
import numpy as np
def K(x):

    return (15/16)*(1-x*x)

def F(x,rend):
    n= len(rend)
    h= 1.06*np.std(rend)*(n**(-1/5))
    somme=0
    for i in range(len(rend)):
        u=(x-rend[i])/h

        if (abs(u)<=1) :
           somme+=K(u)
        else:
            somme+=0

    return somme/len(rend)


def dichotomie(rend,alpha):
    a=-1
    b=1
    eps = 0.000000001
    while abs(b - a) > eps:
        m = (a + b)/2
        if F(m,rend) == 1-alpha:
            return m
        elif F(m,rend) < 1-alpha:
            a = m
        else:
            b = m
    return (a + b) / 2

Var_Nonpara=dichotomie(df['Price Returns'], 0.95)
print('VaR Non-parametric at 95%  =',Var_Nonpara)

#print('1-alpha =',F(Var_Nonpara,df['Price Returns']))
#On travaille sur des pertes donc on va prendre -VaR pour avoir une VaR positive
Var_Nonpara=-Var_Nonpara


VaR Non-parametric at 95%  = -0.030911413487046957


In [25]:
import pandas as pd

with open('/content/sample_data/Natixis stock (dataset TD1&2).txt', 'r') as file:
  lines = file.readlines()


dates =[]
prices = []

for line in lines:
  line=line.strip()
  if line:
    date,price= line.split('\t')
    dates.append(date)
    prices.append(price)

d = {'Date':dates, 'Price':prices}
df=pd.DataFrame(d)

df['Price']= df['Price'].str.replace(',','.').astype(float)
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df.set_index('Date', inplace=True)

start_date= pd.to_datetime('01/01/2017', format='%d/%m/%Y')
end_date = pd.to_datetime('31/12/2018', format='%d/%m/%Y')

df = df.loc[start_date:end_date]

returns =[]
for i in range(1,len(df)):
  price_yesterday= df['Price'][i-1]
  price_today = df['Price'][i]
  returns.append((price_today-price_yesterday)/price_yesterday)

df['Price Returns'] =  [None] + returns
df = df.iloc[1:] #del the Nan values of the first row
df = df.sort_values(by='Price Returns', ascending=True)

In [26]:
exceeding_returns=df[df['Price Returns']>Var_Nonpara]

proportion = len(exceeding_returns)/len(df)

print("La porportion de returns excendent la VaR est de ",100*proportion,"%")

La porportion de returns excendent la VaR est de  2.161100196463654 %


Question B

In [11]:
import pandas as pd

with open('/content/sample_data/Natixis stock (dataset TD1&2).txt', 'r') as file:
  lines = file.readlines()


dates =[]
prices = []

for line in lines:
  line=line.strip()
  if line:
    date,price= line.split('\t')
    dates.append(date)
    prices.append(price)

d = {'Date':dates, 'Price':prices}
df=pd.DataFrame(d)

df['Price']= df['Price'].str.replace(',','.').astype(float)
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df.set_index('Date', inplace=True)

start_date= pd.to_datetime('01/01/2015', format='%d/%m/%Y')
end_date = pd.to_datetime('31/12/2016', format='%d/%m/%Y')

df = df.loc[start_date:end_date]

returns =[]
for i in range(1,len(df)):
  price_yesterday= df['Price'][i-1]
  price_today = df['Price'][i]
  returns.append((price_today-price_yesterday)/price_yesterday)

df['Price Returns'] =  [None] + returns
df = df.iloc[1:] #del the Nan values of the first row
df = df.sort_values(by='Price Returns', ascending=True)

In [28]:
def expected_shortfall(var,data):
  losses = [-losse for losse in data if losse<var ]
  ES= sum(losses)/len(losses)
  return ES


Var_Nonpara=-Var_Nonpara
#print(f"Expcetced shortfall for historical var is : {expected_shortfall(VaR_historical,df['Price Returns'])}")
#print(f"Expcetced shortfall for Parametric_var var is : {expected_shortfall(Parametric_var,df['Price Returns'])}")

print(f"L'Expected shortfall pour la VaR non-parametric est : {expected_shortfall(Var_Nonpara,df['Price Returns'])}")

#print(f"Expcetced shortfall for VaR in BlackScholes is : {expected_shortfall(var,df['Price Returns'])}")

L'Expected shortfall pour la VaR non-parametric est : 0.043994256217435135
